# YANDEX.MUSIC

Comparison of Moscow and St. Petersburg is surrounded by myths. For example:
  * Moscow is a metropolis subject to the rigid rhythm of the working week;
  * St. Petersburg is a cultural capital, with its own tastes.


**Research Purpose** - Test three hypotheses:
1. User activity depends on the day of the week. Moreover, in Moscow and St. Petersburg is manifests itself in different ways.
2. On Monday morning, certain genres prevail in Moscow, while others prevail in St. Petersburg. Also, different genres dominate on Friday evenings, depending on the city.
3. Moscow and St. Petersburg prefer different genres of music. In Moscow, they listen to pop music more often, in St. Petersburg - Russian rap.

**Research Progress**

We received the data `yandex_music_project.csv`. Nothing is known about the quality of the data. Therefore, before testing hypotheses, a review of the data is required.

 
Thus, the study will take place in three stages:
  1. Data review.
  2. Data preprocessing.
  3. Hypothesis testing.

According to the data documentation:
* `userID` - user ID;
* `Track` — track name;
* `artist` — artist name;
* `genre` — genre name;
* `City` - user's city;
* `time` - start time of listening;
* `Day` - the day of the week.

## Data overview





In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('/datasets/yandex_music_project.csv')
df.head()

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


In [11]:
df.describe()

,userID,Track,artist,genre,City,time,Day
count,65079,63848,57876,63881,65079,65079,65079
unique,41748,47245,43605,289,2,20392,3
top,A8AE9169,Intro,Rocket,pop,Moscow,21:51:22,Friday
freq,76,34,6,8850,45360,14,23149


**Conclusion**

So the table has seven columns. The data type in all columns is `object`.



There are three style violations in the column headings:
1. Lowercase letters are combined with uppercase.
2. There are gaps.
3. The snake case is not respected.



The number of values in the columns varies. This means there are missing values in the data.

To move forward, we need to fix problems in the data.

## Data preprocessing


### Стиль заголовков

Firstly we change the column names

In [12]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [13]:
df = df.rename(columns = {'  userID':'user_id', 'Track':'track', '  City  ':'city', 'Day':'day'})
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Missing values


In [14]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Not all missing values affect the study. So in `track` and `artist` the gaps are not important for the job.

But omissions in `genre` can interfere with the comparison of musical tastes in Moscow and St. Petersburg. 

Let's do:
* fill in these gaps with explicit notation,
* estimate how much they will damage the calculations.

In [15]:
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

df.isna().sum()
                                   

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Duplicates


In [16]:
df.duplicated().sum()

3826

We have `3826` explicit duplicates here. We don't need them, so I'm just gonna remove all of them. 

In [17]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

In [21]:
df['genre'].sort_values().unique().tolist()

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'alternativepunk',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'author',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'chanson',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',


In column `genre` I found few duplicates^ such as

* *hip*,
* *hop*,
* *hip-hop*,
* *hiphop*



In [23]:
def replace_wrong_genres(wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genres, correct_genre)

duplicates = ['hip', 'hop', 'hip-hop']
name = 'hiphop'
replace_wrong_genres(duplicates, name)

df['genre'].sort_values().unique().tolist()

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'alternativepunk',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'author',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'chanson',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',


**Выводы**

Now we don't have duplicates and the columns look good

## Hypothesis testing

### Comparison of user behavior of two capitals

The first hypothesis states that users listen to music differently in Moscow and St. Petersburg. Let's check this statement for three days of the week: Monday, Wednesday and Friday.


In [24]:
df.groupby('city').count()

,user_id,track,artist,genre,time,day
city,,,,,,
Moscow,42741,42741,42741,42741,42741,42741
Saint-Petersburg,18512,18512,18512,18512,18512,18512


The time of listening in Moscow is higher than in St. Petersburg. It does not follow from this that Moscow users listen to music more often. There are simply more users in Moscow.



In [25]:
df.groupby('day').count()

,user_id,track,artist,genre,city,time
day,,,,,,
Friday,21840,21840,21840,21840,21840,21840
Monday,21354,21354,21354,21354,21354,21354
Wednesday,18059,18059,18059,18059,18059,18059


On average, users from the two cities are less active on Wednesdays. But the picture may change if we consider each city separately.

In [26]:
def number_tracks(day, city):
    track_list = df.loc[(df['day'] == day) & (df['city'] == city)]
    track_list_count = track_list['user_id'].count()
    return(track_list_count)

In [27]:
number_tracks('Monday', 'Moscow')

15740

In [28]:
number_tracks('Monday', 'Saint-Petersburg')

5614

In [29]:
number_tracks('Wednesday', 'Moscow')

11056

In [30]:
number_tracks('Wednesday', 'Saint-Petersburg')

7003

In [31]:
number_tracks('Friday', 'Moscow')

15945

In [32]:
number_tracks('Friday', 'Saint-Petersburg')

5895

In [33]:
column = ['city', 'monday', 'wednesday', 'friday']
data = [
    ['Moscow', '15740', '11056', '15945'],
    ['Saint-Petersburg', '5614', '7003', '5895']
]
table = pd.DataFrame(data=data, columns=column)
display(table)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Conclusion**

The data shows the difference in user behavior:

- In Moscow, the peak of listening falls on Monday and Friday, and on Wednesday there is a noticeable decline.
- In St. Petersburg, on the contrary, they listen to music more on Wednesdays. Activity on Monday and Friday here is almost equally inferior to Wednesday.

So the data support the first hypothesis.

### Music at the beginning and the end of the week

According to the second hypothesis, on Monday morning certain genres predominate in Moscow, while others dominate in St. Petersburg. Also, Friday evenings are dominated by different genres, depending on the city.

In [34]:
moscow_general = df.loc[df['city'] == 'Moscow']
display(moscow_general)

,user_id,track,artist,genre,city,time,day
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,unknown,ruspop,Moscow,09:17:40,Friday
...,...,...,...,...,...,...,...
61247,83A474E7,I Worship Only What You Bleed,The Black Dahlia Murder,extrememetal,Moscow,21:07:12,Monday
61248,729CBB09,My Name,McLean,rnb,Moscow,13:32:28,Wednesday
61250,C5E3A0D5,Jalopiina,unknown,industrial,Moscow,20:09:26,Friday
61251,321D0506,Freight Train,Chas McDevitt,rock,Moscow,21:43:59,Friday


In [35]:
spb_general = df.loc[df['city'] == 'Saint-Petersburg']
display(spb_general)

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
9,E772D5C0,Pessimist,unknown,dance,Saint-Petersburg,21:20:49,Wednesday
...,...,...,...,...,...,...,...
61239,D94F810B,Theme from the Walking Dead,Proyecto Halloween,film,Saint-Petersburg,21:14:40,Monday
61240,BC8EC5CF,Red Lips: Gta (Rover Rework),Rover,electronic,Saint-Petersburg,21:06:50,Monday
61241,29E04611,Bre Petrunko,Perunika Trio,world,Saint-Petersburg,13:56:00,Monday
61242,1B91C621,(Hello) Cloud Mountain,sleepmakeswaves,postrock,Saint-Petersburg,09:22:13,Monday


In [36]:
def genre_weekday(table, day, time1, time2):
    genre_df = table.loc[(table['day'] == day) & (table['time'] > time1) & (table['time'] < time2)]
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending = False)
    return(genre_df_sorted.head(10))

In [37]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [38]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [39]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [40]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Conclusion**

If we compare the top 10 genres on Monday morning, we can draw the following conclusions:

1. In Moscow and St. Petersburg they listen to similar music. The only difference is that the Moscow rating includes the “world” genre, while the St. Petersburg rating includes jazz and classical.

2. There were so many missing values ​​in Moscow that the value `'unknown'` took tenth place among the most popular genres. This means that missing values ​​occupy a significant share in the data and threaten the reliability of the study.

Friday night does not change this picture. Some genres rise a little higher, others go down, but overall the top 10 stays the same.

Thus, the second hypothesis was only partially confirmed:
* Users listen to similar music at the beginning of the week and at the end.
* The difference between Moscow and St. Petersburg is not very pronounced. In Moscow, they listen to Russian popular music more often, in St. Petersburg - jazz.

However, gaps in the data cast doubt on this result. There are so many of them in Moscow that the top 10 ranking could look different if it were not for the lost genre data.

### Genre preferences in Moscow and St. Petersburg

Hypothesis: St. Petersburg is the capital of rap, the music of this genre is listened to more often than in Moscow. And Moscow is a city of contrasts, which, nevertheless, is dominated by pop music.

In [41]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending = False)
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

In [42]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending = False)
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Conclusion**

The hypothesis was partially confirmed:
* Pop music is the most popular genre in Moscow, as the hypothesis suggested. Moreover, in the top 10 genres there is a close genre - Russian popular music.
* Contrary to expectations, rap is equally popular in Moscow and St. Petersburg.

## Research results

We tested three hypotheses and found:

1. The day of the week has a different effect on the activity of users in Moscow and St. Petersburg.

The first hypothesis was fully confirmed.

2. Musical preferences do not change much during the week - be it Moscow or St. Petersburg. Small differences are noticeable at the beginning of the week, on Mondays:
* in Moscow they listen to music of the “world” genre,
* in St. Petersburg - jazz and classical music.

Thus, the second hypothesis was only partly confirmed. This result could have been different were it not for gaps in the data.

3. The tastes of users of Moscow and St. Petersburg have more in common than differences. Contrary to expectations, genre preferences in St. Petersburg resemble those in Moscow.

The third hypothesis was not confirmed. If there are differences in preferences, they are invisible to the bulk of users.
